<a href="https://colab.research.google.com/github/notsurez/GolfCaddie/blob/main/Senior_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 497.5 MB 27 kB/s 
     |████████████████████████████████| 462 kB 64.1 MB/s 


In [ ]:
!nvidia-smi

Wed Apr 13 02:28:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# New Section

In [ ]:

from google.colab import files
 
 
uploaded = files.upload()

Saving trainingdata.csv to trainingdata.csv


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
import math
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

## **Load our data into the colab notebook**

In [ ]:
import numpy as np

with open("trainingdata.csv") as file_name:
    train_data = np.loadtxt(file_name, delimiter=",")

test_data = train_data;
test_data.shape
test_data = np.delete(test_data,slice(20000,100000,1), axis = 0)
train_data = np.delete(train_data,slice(0,20000,1),axis = 0)


## **Normalize our data**

In [ ]:
mean_array = np.mean(train_data,axis = 0)
meantest = np.mean(test_data,axis = 0)
for x in range(0, 26):
  train_data[:,x] -= mean_array[x]
  test_data[:,x] -= meantest[x]
std_array = np.std(train_data,axis = 0)
stdtest = np.std(test_data,axis = 0)
for x in range(0, 26):
  train_data[:,x] /= std_array[x]
  test_data[:,x] /= stdtest[x]



[[ 5.71572784e-01 -1.89879856e+00  1.03296735e+00 ... -5.72163420e-01
  -1.38820224e+00  5.28981723e+00]
 [-9.67483903e-01  7.95159472e-01  2.03702753e+00 ... -2.79843587e-01
   4.94404847e-03  5.62041168e+00]
 [-7.08690801e-01 -1.62334264e+00 -4.44714216e-01 ...  4.66822471e-01
  -8.91195454e-01  4.18785488e+00]
 ...
 [-1.56041824e+00  1.38003242e+00  1.87085270e+00 ... -1.08880987e+00
   1.93727663e-01  4.70877257e+00]
 [ 6.21443167e-01  4.46790709e-01 -1.62648334e+00 ...  3.43813392e-02
   1.77420748e+00  4.19630920e+00]
 [ 1.06784780e+00  6.34361399e-01  9.19246605e-01 ...  1.14860927e+00
   9.51541410e-01  3.45457306e+00]]


## **Build Our Model**

In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Dense(16, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(2, activation="softmax")
    ])
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    return model

# ***K-Fold Validation***

In [ ]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=16, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

Processing fold #0


NameError: ignored

In [ ]:
all_scores

In [ ]:
np.mean(all_scores)

## ***Saving the validation logs at each fold***

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=16, verbose=0)
    mae_history = history.history["val_mae"]
    all_mae_histories.append(mae_history)

### ***Building the history of successive mean K-fold validation scores***

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

### ***Plotting the validation scores***

In [ ]:
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

### ***Plotting validation scores, excluding the first 10 data points***

In [ ]:
truncated_mae_history = average_mae_history[10:]
plt.plot(range(1, len(truncated_mae_history) + 1), truncated_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

## ***Training the final model***

In [ ]:
model = build_model()
model.fit(train_data, train_targets,
          epochs=130, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score

### ***Generating predictions on new data***

In [ ]:
predictions = model.predict(test_data)
predictions[0]